# Diseño de un procesador RISC-V (ALGOL)

Ángel Terrones

# RISC-V
RISC-V es un conjunto de instrucciones (ISA) creado con el fin de ser usado en eduación e investigación. Desarrollado por la División de Ciencias de la Computación del Departamento de Electrónica y Computación de la Universidad de California, Berkeley (UCB).

# Especificaciones del ISA (RV32I)



## Programmers' Model
* Sistema de memoria __litte-endian__.
* 32 registros de propósito general.
    * El registro cero está _hardwired_ a __cero__
* 4 formatos de instrucciones __base__:
    * R
    * I
    * S
    * U
* La ubicación de las fuentes de operandos __son fijas__: _rs1_ y _rs2_.
* La ubicación del registro destino __es fija__: _rd_.
* Variante para valores inmediatos:
    * SB
    * UJ
* Armar los valores de inmediatos es engorroso (ver __página 12__ del __Volúmen I__ del ISA).

## Set de Intrucciones Base de 32 bits (RV32I)
(El RV32I también está relacionado con el modo de ejecución de usuario).
Existe un total de 37 instrucciones base + 10 relacionadas con acceso (reducido) al sistema.
* Cálculo con enteros:
    * Register-Immediate.
    * Register-Register.
* Transferencia de control:
    * Saltos incondicionales.
    * Saltos condicionales.
* Acceso a memoria:
    * Load B/BU/H/HU/W
    * Store B/H/W
* Soporte básico para acceso a la memoria en forma atómica:
    * FENCE
    * FENCE.I
* Instrucciones de sistema:
    * SCALL
    * SBREAK
* Acceso a _timers_ y _contadores_

## Lista y formato de las instrucciones RV32I


![BSOD](img/windows-10-bsods-on-asus-pcs-faulty-hardware-corrupted-page-error-492252-2.jpg)

(Muy larga para copiar. Ver resumen en la __página 50__ del __Volúmen I__ del ISA).

# Especificaciones del Modo Privilegiado.


## Modos de ejecución
Existen 4 modos:
* User (U)
* Supervisor (S)
* Hypervisor (H)
* Machine (M)

__Obligatoriamente__ debe ser implementado el modo __M__. En este modo, el privilegio de ejecución es máximo.

## Instrucciones del sistema
La lista de instrucciones a implementar para el modo privilegiado se encuentran descritas en la __página 52__.
* Excepción de esta lista: 
    * WFI
    * SFENCE.VM.
    * MRTS
    * MRTH
    * HRTS
* Pueden ser implementatas como NOPs.

## Control and Status Register (CSR)

Este bloque/módulo no es más que un __register file__. 
Este bloque abarca un conjunto de registros que, como su nombre indica, sirven para indicar la configuración del procesador. Algunos de estos registros solo pueden ser accesados (RO, R/W) con un nivel de privilegio mínimo. En caso contrario de debe generar una __excepción__.

Una lista con direcciones accesibles para cada modo se encuentra disponible en las __páginas 10-13__ del __Volúmen II__

## Las intrucciones de acceso al CSR
* Leen el valor actual del registro del CSR, y lo almacenan en un registro destino __rd__ en el _register file_.
* Actualizan el registro del CSR según el tipo de instrucción:
    * Reemplazo completo: CSRRW(I)
    * Colocar en 1 ciertos bits según una máscara: CSRRS(I)
    * Colocar en 0 ciertos bits según una máscara: CSRRC(I)
* Las intrucciones anteriores no __actualizan el registro del CSR__ si:
    * El campo __rs1__ es cero.
    Esto implica que la operación de escritura en CSR no se realiza, pero la de lectura si.

# Diseño del procesador ALGOL (RV32IM)




![Algol](img/perseus1.jpg "The Demon Star")

# Especificaciones
* Implementación del ISA __RV32IM__.
* Segmentado de 5 etapas.
* _Forwarding_.
* Soporte para modo __UM__ (User and Machine)
* Sin predictor de saltos (TODO)
* Sin cache (TODO)
* Sin memoria virtual (TODO)

# Lista de componentes necesarios

* Register File
* ALU
* Generador de inmediato.
* Multiplexores 4-1
* Multiplexores 2-1
* CSR
* Registros del pipeline:
    * PC
    * IF->ID
    * ID->EX
    * EX->MEM
    * MEM->WB
* Control Unit
    * Instruction Decoder.
    * Branch detection.
    * Hazard detection.
    * Pipeline control.
    * Interfaz con memoria.

# Register File
![RF](img/register_file.gif "Register File")

* Implementada como una pequeña memoria de 32 words.
* Dos puertos de lectura.
* Un puerto de escritura.
* Tomado del proyecto [__ANTARES__](https://github.com/AngelTerrones/Antares) (procesador MIPS de 32 bits).

# ALU
* Versión simple implementa:
    * 2 puertos de entrada de datos.
    * 1 puerto de salida.
    * Puerto para control (función/opcode).
* Funciones implementadas:
    * Aritméticas: +, -
    * Lógica: &, |, ^
    * Shift: LL, LR, AR
    * Comparación: SLT, SLTU

![ALU](img/alu.png)

# CSR
* ~~Copiado~~Basado en el CSR del proyecto [VScale](https://github.com/ucb-bar/vscale) (RV32I segmentado de 3 etapas, implementado en verilog).

* Implementados todos los registros del nivel de usuario.
* Implementados todos los registros del nivel de máquina.
    * Incluyendo el registro __mtohost__ el cual es necesario para los tests.

* El CSR se encarga del manejo del estado del procesador (priv level):
    * Input: exception flag.
    * Input: exception code.
    * Input: ERET instruction.
    * Input: Bad address for LD/ST.
    * Input: PC de la instrucción que __genera__ el fallo.
    * Output: PC handler.
    * Output: EPC.

* El CSR se encarga de la detección de interrupciones:
    * Output: Interrupt flag.
    * Output: Interrupt code.

# Exceptions
¿Dónde detectar y procesar las excepciones?

## Detectar excepción

La detección se hace en las etapas que pueden generar la excepción:
* IF
* ID
* MEM

salvo que se requiera alguna información extra, o que dependa de la completa ejecución de las instrucciones anteriores.
En tal caso, se retrasa la detección hasta la etapa __MEM__.
Ejemplo:
* ERET.
* System/Environment calls.

## Procesar las excepciones


![KISS](img/keep-it-simple-blogger.jpg)

Procesar todas las excepciones en la última etapa que las puede generar: __MEM__. Tiene la desventaja de añadir latencia fija para las exceptiones generadas en __IF__ e __ID__.

Consideraciones:
* Implica propagar las señales de excepciones desde __IF__ hasta __MEM__.
* Se debe tener cuidado de que instrucciones que generen excepciones no generen nuevas excepciones al transitar por el pipeline.
    * En caso de simplificar el control, en necesario mantener la prioridad por histórico: IF > ID > MEM.
* El modo privilegiado actual solo es válido en la etapa __MEM__. Por lo que instrucciones que requieren verificar o dependen del modo, deben esperar hasta __MEM__ para realizar el check y hacer _commit_.
    * Esto aplica para las instrucciones CSRXY. Sin embargo, el CSR puede (¿__debe__?) detectar accesos ilegales y generar una excepcion.
* Al generar una excepcion, el pipeline completo debe ser _reiniciado_, aunque ya esto es __obvio__.

# Verificar el procesador

¿Para qué?


![BUGS EVERYWHERE](img/e52572d331528e9dbcfba78277c7dde1.jpg)

![Bugs](img/1743673_10152300393504108_165443790_n.jpg)
Hay que tener cuidado al reparar problemas en el código, para evitar insertar más errores en el diseño.

Metodología:
* Probar el procesador.
* Detectar fallas.
* Reparar fallas.
* Volver a probar el procesador (desde cero, incluyendo test que ya funcionaron).
* Detectar fallas.
* Repetir hasta cumplir con el 100% de las prueba.

# Plataforma de pruebas.

* Realizada en python.
* Se conecta con el procesador usando una memoria de prueba.
* Las pruebas estan basadas en programas escritos en ensamblador por los mismos creadores del procesaror [Rocket](https://github.com/ucb-bar/rocket/). Las pruebas están en el repositorio [riscv-test](https://github.com/riscv/riscv-tests)

# Demo time

![Demo](img/62601759.jpg)